In [ ]:
import warnings
import os
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score
from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from xgboost import plot_importance
from lightgbm import LGBMClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import eli5
from eli5.sklearn import PermutationImportance
from pystacknet.pystacknet import StackNetClassifier
from imblearn.combine import SMOTETomek
import plotly.graph_objects as go

os.environ['CUDA_VISIABLEDIVCES']='4'

In [ ]:
df = pd.read_csv('../featureData/32_feature/first_GLCM/GLCM.csv')

In [ ]:
Ran_state = 25

In [ ]:
df.info

데이터 불러오기 끝.
데이터 전처리 시작

In [ ]:
data_y = df['label']

data_x = df.drop('label',1)
# print(data_y)
# print(data_x)

연속형 변수들 MinMaxScaler 진행

In [ ]:
########Why these indexes are dropped? => 키나 몸무게외 같은 연속성 수치이기 때문에 scaling을 하지 않음
DFS = data_x.loc[:,['First_Order_10Percentile','First_Order_90Percentile','First_Order_Energy',	'First_Order_InterquartileRange',
                    'First_Order_Maximum','First_Order_MeanAbsoluteDeviation',	'First_Order_Mean',	'First_Order_Median','First_Order_Minimum',
                    'First_Order_Range','First_Order_RobustMeanAbsoluteDeviation','First_Order_RootMeanSquared','First_Order_TotalEnergy',	
                    'First_Order_Variance','GLCM_Autocorrelation','GLCM_ClusterProminence','GLCM_SumAverage'
]]
data_x = data_x.drop(DFS.columns,1)
# print(data_x)

In [ ]:
#########연속형 변수를 제외하고 스케일링을 진행한 후, 제외된 연속된 변수를 다시 포함해줌(feature scaling)
#########현 데이터는 필요 없음===> first_ortder의 energy와 total energy의 수치는 매우 크므로 적용
MM = MinMaxScaler()
DFSMM = MM.fit_transform(DFS)
DFS = pd.DataFrame(DFSMM,columns=DFS.columns)
data_x = pd.concat([DFS, data_x],1)
# data_x.to_csv('./Energy_scaled_all.csv', sep=',')

In [ ]:
print(data_x)

다중공선성 높은 변수들 제거

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["features"] = data_x.columns
vif["VIF Factor"] = [variance_inflation_factor(data_x.values, i) for i in range(data_x.shape[1])]
vif_remove = vif.sort_values(ascending=False,by='VIF Factor')[:]
vif_remove.to_csv('../Radiomics/sorted_vif_GLCM.csv', sep=',')

In [ ]:
vif_remove

In [ ]:
####10이상의 수치를 가진 것들 제외(하나만 남기고 제외)=>모든 feature가 10미만이 될 때까지?
# data_x_outEnergy = data_x.drop(['First_Order_Energy'],1)
data_x_out = data_x.drop(['GLCM_ClusterTendency','GLCM_JointAverage','First_Order_Energy','First_Order_Maximum','GLCM_Id','GLCM_Idmn','GLCM_Idn','GLCM_Idm','GLCM_JointEntropy',
                          'GLCM_SumEntropy','First_Order_Mean','GLCM_DifferenceAverage','First_Order_RootMeanSquared','GLCM_Imc2','First_Order_Uniformity',
                          'GLCM_DifferenceEntropy','GLCM_Correlation','First_Order_Entropy','GLCM_SumSquares','First_Order_MeanAbsoluteDeviation','First_Order_90Percentile',
                         'GLCM_SumAverage','GLCM_MCC','First_Order_Median','GLCM_Contrast','First_Order_10Percentile','First_Order_Variance','First_Order_TotalEnergy','GLCM_JointEnergy',
                         'First_Order_Range','First_Order_RobustMeanAbsoluteDeviation','GLCM_InverseVariance','GLCM_Imc1','GLCM_DifferenceVariance'],1)

In [ ]:
# print(data_x_outEnergy)
vif = pd.DataFrame()
vif["features"] = data_x_out.columns
vif["VIF Factor"] = [variance_inflation_factor(data_x_out.values, i) for i in range(data_x_out.shape[1])]
vif_remove = vif.sort_values(ascending=False,by='VIF Factor')[:]
vif_remove.to_csv('../Radiomics/sorted_vif_out.csv', sep=',')
vif_remove

In [ ]:
vif_remove['VIF Factor'][22].dtype

In [ ]:
print(vif_remove['features'])

In [ ]:
nan=[]
for vf in range(len(vif_remove)):
    if vif_remove['VIF Factor'][vf]<=0:
        print(vif_remove['features'][vf])
        nan.append(vif_remove['features'][vf])

In [ ]:
print(nan)

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(data_x_out,data_y,test_size=0.3,random_state = Ran_state)

In [ ]:
#######dropout된 값을 train_x로 

In [ ]:
print(train_x)

하이퍼 파라미터 조정용 GridsearchCV 진행

In [ ]:
LR_grid={'penalty': ['l1', 'l2'], 
            'C': [0.001, 0.01, 0.1, 1, 10, 100],'random_state' : [Ran_state]
                   }

RF_grid={'max_depth': [4,6,8,10,12,14], # max_depth: The maximum depth of the tree.
              'n_estimators': [100,200,400,600], # n_estimators: The number of trees in the forest.
              'min_samples_split': [50, 100,25,75,125],'random_state' : [Ran_state]
              }

XGB_grid={'n_estimators' : [200,400,600],
              'learning_rate' : [0.01,0.05,0.10,0.15],
              'max_depth' : [4,6,8,10,12],'random_state' : [Ran_state]} 

SVM_grid = {'gamma': [0.01, 0.1, 1, 10, 100], 
              'C': [0.01, 0.1, 1, 10, 100],'random_state' : [Ran_state]}

LR = LogisticRegression()
RF = RandomForestClassifier()
XGB = XGBClassifier()
SVM = SVC()

gridlist = [LR_grid,RF_grid,XGB_grid,SVM_grid]
classlist = [LR,RF,XGB,SVM]

###GridSearchCV로 파라미터 튜닝을 위해 파라미터값 지정

In [ ]:
def gridsearch(gridlist,classlist,train_x,train_y):
    for i in range(4):
        grid = GridSearchCV(classlist[i], gridlist[i], scoring = 'roc_auc', cv=7,n_jobs=-1)  
        grid.fit(train_x, train_y)
        print(grid.best_params_)
        print(grid.best_score_)
        classlist[i] = grid.best_estimator_
    return classlist

In [ ]:
classlist = gridsearch(gridlist,classlist,train_x,train_y)

==================================================================

학습 및 예측

In [ ]:
train_x.values

In [ ]:
def get_stacking_data(classlist,train_x,train_y,test_x,n_folds=5): #스태킹 앙상블 모델
#     train_x = train_x.as_matrix()
#     test_x = test_x.as_matrix()
    
    train_x = train_x.values     #########
    test_x = test_x.values       #########
    
    test_x = test_x.astype(float)
    train_x = train_x.astype(float)
    models=[classlist,
            [LGBMClassifier(n_estimators=400)],
            ]
    stk=StackNetClassifier(models, metric="auc", folds=n_folds,
        restacking=False,use_retraining=True, use_proba=True, 
        random_state=Ran_state,n_jobs=-1, verbose=1)
    stk.fit(train_x,train_y)
    preds=stk.predict_proba(test_x)[:,1]
    return preds

In [ ]:
def create_curve(classlist,train_x,train_y,test_x,test_y,save_path):
    LR = classlist[0]
    RF = classlist[1]
    XGB = classlist[2]
    SVM = classlist[3]

    LR.fit(train_x,train_y)
    RF.fit(train_x,train_y)
    XGB.fit(train_x,train_y)
    SVM.fit(train_x,train_y)
    
    print('==========================================')
    print(train_x.columns)
    print(LR.coef_)
    print('==========================================')

    lr_proba = LR.predict_proba(test_x)[:,1]
    rf_proba = RF.predict_proba(test_x)[:,1]
    xgb_proba = XGB.predict_proba(test_x)[:,1]
    SVM_proba = SVM.decision_function(test_x)
    lgbm_proba = get_stacking_data(classlist,train_x,train_y,test_x,5)       #######################

    fpr_lr,tpr_lr,_=roc_curve(test_y,lr_proba)
    fpr_rf,tpr_rf,_=roc_curve(test_y,rf_proba)
    fpr_xgb,tpr_xgb,_=roc_curve(test_y,xgb_proba)
    fpr_svm,tpr_svm,_ = roc_curve(test_y,SVM_proba)
    fpr_lgbm,tpr_lgbm,_=roc_curve(test_y,lgbm_proba)

    auc_lr=auc(fpr_lr, tpr_lr)
    auc_xgb=auc(fpr_xgb, tpr_xgb)
    auc_rf=auc(fpr_rf, tpr_rf)
    auc_svm=auc(fpr_svm, tpr_svm)
    auc_lgbm=auc(fpr_lgbm, tpr_lgbm)

    plt.plot(fpr_lr, tpr_lr, label="LR (AUC= %s)"%(round(auc_lr,3)))
    plt.plot(fpr_rf, tpr_rf, label="RF (AUC= %s)"%(round(auc_rf,3)))
    plt.plot(fpr_xgb, tpr_xgb, label="XGB (AUC= %s)"%(round(auc_xgb,3)))
    plt.plot(fpr_svm, tpr_svm, label="SVM (AUC= %s)"%(round(auc_svm,3)))
    plt.plot(fpr_lgbm, tpr_lgbm, label="S.ESB (AUC= %s)"%(round(auc_lgbm,3)))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('1-specificity',fontsize=12)
    plt.ylabel('sensitivity',fontsize=15)
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right",fontsize=12)
    plt.show()
    
    Importance = pd.DataFrame(train_x.columns,columns=['feature'])
    suf = ('_LR','_RF','_XGB','_SVM')
    
    for i in range(4):
        PI = P_Importance(classlist[i],test_x,test_y).loc[:,['feature','weight']]
        Importance = pd.merge(Importance,PI,on='feature',suffixes=(suf[i-1],suf[i]))
    
    Importance['meanscore'] = Importance.apply(lambda x: np.mean(x[1:5]),1)
    ftr_top20 = Importance.sort_values(ascending=False,by='meanscore')[:]
    
    fig = go.Figure([go.Bar(x=ftr_top20.meanscore, y=ftr_top20.feature,orientation='h',marker={'color': ftr_top20.meanscore,'colorscale': 'Oryel'})])
    fig.update_layout(
    autosize=False,
    width=500,
    height=800,
    yaxis=dict(type='category',tickfont_size=18)
    
    )
    
    
    fig.show()
    plt.savefig(save_path, dpi=300, facecolor='w', edgecolor='w')       ###############
    
    return Importance

In [ ]:
# importance

def P_Importance(model,test_x,test_y): #permutation importance 중요도 뽑기 
    perm = PermutationImportance(model, scoring = "roc_auc", random_state = Ran_state).fit(test_x, test_y) 
    ftr_importances = eli5.format_as_dataframe(eli5.explain_weights(perm,top=134, feature_names = test_x.columns.tolist()))
    ftr_top = ftr_importances.sort_values(ascending=False,by='feature')[:]
    return ftr_top

In [ ]:
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y, './featureData/importanced_roc_1219')

In [ ]:
ftr_importances

위에서 뽑은 importance 이용 Feature selection 과정

In [ ]:
def roc_score(classlist,train_x,train_y,test_x,test_y):
    LR = classlist[0]
    RF = classlist[1]
    XGB = classlist[2]
    SVM = classlist[3]
    
    LR.fit(train_x,train_y)
    RF.fit(train_x,train_y)
    XGB.fit(train_x,train_y)
    SVM.fit(train_x,train_y)
    
    lr_proba = LR.predict_proba(test_x)[:,1]
    rf_proba = RF.predict_proba(test_x)[:,1]
    xgb_proba = XGB.predict_proba(test_x)[:,1]
    SVM_proba = SVM.decision_function(test_x)
    
    lr_score = roc_auc_score(test_y, lr_proba)
    rf_score = roc_auc_score(test_y, rf_proba)
    xgb_score = roc_auc_score(test_y, xgb_proba)
    SVM_score = roc_auc_score(test_y, SVM_proba)
    
    print(lr_score,rf_score,xgb_score,SVM_score)
    
    Importance = pd.DataFrame(train_x.columns,columns=['feature'])
    suf = ('_LR','_RF','_XGB','_SVM')
    
    for i in range(4):
        PI = P_Importance(classlist[i],test_x,test_y).loc[:,['feature','weight']]
        Importance = pd.merge(Importance,PI,on='feature',suffixes=(suf[i-1],suf[i]))
    
    Importance['meanscore'] = Importance.apply(lambda x: np.mean(x[1:5]),1)
    
    
    return np.max([lr_score,rf_score,xgb_score,SVM_score]),Importance

In [ ]:
def featurefind(ftr_importances,data_x,data_y,classlist):
    best_score = 0
    for i in range(len(ftr_importances.feature)-2):
        ftr_bot = ftr_importances.sort_values(ascending=True,by='meanscore')[:1]
        data_x = data_x.drop(ftr_bot.feature ,1)
        train_x,test_x,train_y,test_y = train_test_split(data_x,data_y,test_size=0.3,random_state = 25)
        meanscore,ftr_importances = roc_score(classlist,train_x,train_y,test_x,test_y)
        print(i)
        if meanscore>=best_score:
            best_score = meanscore
            best_bot = data_x
            print("best_score = ",best_score)
    return best_bot

In [ ]:
#초회 실행용
new_data_x = featurefind(ftr_importances,data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y,'./featureData/selected_roc1_2020')

In [ ]:
print(ftr_importances)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y,'./featureData/selected_roc2_1219')

In [ ]:
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

남은 변수가 너무 많다싶으면 반복용 코드 반복실행

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y,'./featureData/selected_roc2_1219')

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
#반복용
new_data_x = featurefind(ftr_importances,new_data_x,data_y,classlist)
train_x,test_x,train_y,test_y = train_test_split(new_data_x,data_y,test_size=0.3,random_state = Ran_state)
classlist = gridsearch(gridlist,classlist,train_x, train_y)
#train_x, train_y = SMOTETomek(random_state=25).fit_sample(train_x, train_y)
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)

In [ ]:
ftr_importances = create_curve(classlist,train_x,train_y,test_x,test_y)